# <div align="center"><b> YOLOV11 - PROYECTO FINAL </b></div>

<div align="right">📝 <em><small><font color='Gray'>Nota:</font></small></em></div>

<div align="right"> <em><small><font color='Gray'> La funcionalidad de visualización de jupyter notebooks en <a href="https://github.com/" target="_blank">github</a> es solamente un preview.</font></small></em> </div>

<div align="right"> <em><small><font color='Gray'> Para mejor visualización se sugiere utilizar el visualizador recomendado por la comunidad: <a href="https://nbviewer.org/" target="_blank">nbviewer</a></font></small></em> </div>

<div align="right"> <em><small><font color='Gray'> Puedes a acceder al siguiente enlace para ver este notebook en dicha página: <a href="https://nbviewer.org/ruta/de/archivo.ipynb">Ruta archivo</a></font></small></em> </div>

* * *

<style>
/* Limitar la altura de las celdas de salida en html */
.jp-OutputArea.jp-Cell-outputArea {
    max-height: 500px;
}
</style>

🛻 <em><font color='MediumSeaGreen'>  Instalaciones: </font></em> 🛻

Este notebook utiliza [Poetry](https://python-poetry.org/) para la gestión de dependencias.
Primero instala Poetry siguiendo las instrucciones de su [documentación oficial](https://python-poetry.org/docs/#installation).
Luego ejecuta el siguiente comando para instalar las dependencias necesarias y activar el entorno virtual:

- Bash:
```bash
poetry install
eval $(poetry env activate)
```

- PowerShell:
```powershell
poetry install
Invoke-Expression (poetry env activate)
```

<!-- Descargar archivos adicionales:
!gdown https://drive.google.com/drive/folders/1UBZ8PEbtmiWMGkULu7GAt3VhUpeTy9l7?usp=sharing --folder -->

In [1]:
# Chequear versión de CUDA
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Wed_Oct_30_01:18:48_Pacific_Daylight_Time_2024
Cuda compilation tools, release 12.6, V12.6.85
Build cuda_12.6.r12.6/compiler.35059454_0


In [2]:
# Chequear más datos sobre la GPU
!nvidia-smi

Mon May 19 17:49:41 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.36                 Driver Version: 566.36         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4080 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
|  0%   37C    P8              5W /  320W |     499MiB /  16376MiB |      7%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

✋ <em><font color='DodgerBlue'>Importaciones:</font></em> ✋

In [ ]:
import sys, os, json, uuid
from dotenv import load_dotenv

load_dotenv("../.env.dev")

sys.path.append(os.path.abspath("../"))  # Se agrega utilidades generales del modulo IA
sys.path.append(os.path.abspath("../../modulo-apps"))  # Se agrega modulo-mini-apps

from typing import List, Dict, Any, Optional
from pathlib import Path

import ultralytics
from ultralytics import YOLO
import torch


import fiftyone as fo

from apps_utils.logging import Logging
from apps_config.settings import Config
from apps_com_db.mongodb_client import MongoDB

import ia_utils.dataset_utils as DatasetUtils
import ia_utils.gpu_utils as GPUUtils

from apps_com_s3.procesador_s3 import ProcesadorS3
import apps_etiquetado.procesador_anotaciones_mongodb as ProcesadorCocoDataset
import apps_etiquetado.procesador_anotaciones_coco_dataset as UtilsCocoDataset

🔧 <em><font color='tomato'>Configuraciones:</font></em> 🔧


In [4]:
# Crear instancia de Config
CONFIG = Config().config_data
LOGGER = Logging().logger
DB = MongoDB().db

download_folder = Path(CONFIG["folders"]["download_folder"])
DOWNLOAD_RAW_DATASET_FOLDER = Path(CONFIG["folders"]["raw_dataset_folder"])
DOWNLOAD_YOLO_DATASET_FOLDER = Path(CONFIG["folders"]["yolo_dataset"])
DOWNLOAD_PREDICTION_FOLDER = download_folder / "predictions"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"  # Establece el dispositivo.

# Parámetros
BATCH_SIZE = 10  # Tamaño del batch
N_EPOCHS = 10  # Número de épocas
VERBOSE = True  # Muestra época a época la evolución
RANDOM_SEED = 42  # Semilla para la aleatoriedad

print(f"Dispositivo actual: {DEVICE}")
ultralytics.checks()  # Verifica la instalación de ultralytics

Ultralytics 8.3.140  Python-3.13.3 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4080 SUPER, 16376MiB)
Setup complete  (20 CPUs, 127.9 GB RAM, 831.1/1862.9 GB disk)


<div align="center">✨Datos del proyecto:✨</div>

<p></p>

<div align="center">

| Subtitulo       | YoloV11 - Detección de palmeras                                                                                                                |
| --------------- | -------------------------------------------------------------------------------------------------------------------------------------- |
| **Descrpción**  | Notebook de detección de palmeras utilizando YoloV11                                                                                                          |
| **Integrantes** | Bruno Masoller (brunomaso1@gmail.com)                                                                                                  |

</div>

## Descarga del dataset

In [5]:
# DatasetUtils.download_full_dataset()

## Analisis de datos

### Convertir todas las clases a una sola

In [6]:
coco_anotations = UtilsCocoDataset.load_annotations_from_file(DOWNLOAD_RAW_DATASET_FOLDER / "labels.json")
coco_annotations_one_class = UtilsCocoDataset.parse_class_annotations_to(coco_anotations, "palmera")

with open(DOWNLOAD_RAW_DATASET_FOLDER / "labels.json", "w") as f:
    json.dump(coco_annotations_one_class, f, indent=4)
    LOGGER.debug(f"Se guardó el archivo de etiquetas en {DOWNLOAD_RAW_DATASET_FOLDER / 'labels.json'}")

### Cargamos el dataset

> 📝 <em><font color='Gray'>Nota:</font></em> Si hay problema de versiones con `FiftyOne`, borrar la base de datos del mismo que se encuentra en: `C:\Users\<usuario>\.fiftyone\`</em>

In [7]:
dataset_name = "deteccion-palmeras"
dataset = fo.Dataset.from_dir(
    dataset_type=fo.types.COCODetectionDataset,
    dataset_dir=DOWNLOAD_RAW_DATASET_FOLDER,
    overwrite=True,
    name=dataset_name,
)

 100% |███████████████████| 84/84 [360.7ms elapsed, 0s remaining, 232.9 samples/s]      


2025-05-19 16:45:20,855 - eta.core.utils - INFO - _draw -  100% |███████████████████| 84/84 [360.7ms elapsed, 0s remaining, 232.9 samples/s]      


### Observamos el dataset

In [8]:
session = fo.launch_app(dataset, auto=False)

Session launched. Run `session.show()` to open the App in a cell output.


2025-05-19 16:45:29,708 - fiftyone.core.session.session - INFO - launch_app - Session launched. Run `session.show()` to open the App in a cell output.


Accedemos a la visualización: <a>localhost:5151</a>

## Convertir dataset a formato YOLO
<small><em>https://docs.ultralytics.com/reference/cfg/__init__/</em></small>

In [9]:
Path(DOWNLOAD_YOLO_DATASET_FOLDER).mkdir(parents=True, exist_ok=True)
dataset.export(
    export_dir=str(DOWNLOAD_YOLO_DATASET_FOLDER),
    dataset_type=fo.types.YOLOv5Dataset,
    overwrite=True,
    split="full",
)

Found multiple fields ['detections', 'segmentations'] with compatible type (<class 'fiftyone.core.labels.Detections'>, <class 'fiftyone.core.labels.Polylines'>); exporting 'detections'


2025-05-19 16:45:30,410 - fiftyone.core.collections - INFO - _get_matching_label_field - Found multiple fields ['detections', 'segmentations'] with compatible type (<class 'fiftyone.core.labels.Detections'>, <class 'fiftyone.core.labels.Polylines'>); exporting 'detections'


 100% |███████████████████| 84/84 [517.5ms elapsed, 0s remaining, 162.3 samples/s]      


2025-05-19 16:45:30,930 - eta.core.utils - INFO - _draw -  100% |███████████████████| 84/84 [517.5ms elapsed, 0s remaining, 162.3 samples/s]      


Ejemplo con views splits:

```python
for split in splits:
    split_view = dataset_or_view.match_tags(split)
    split_view.export(
        export_dir=export_dir,
        dataset_type=fo.types.YOLOv5Dataset,
        label_field=label_field,
        split=split,
        classes=classes,
    )
```

Dividimos el conjunto de datos en train y test:

In [10]:
DatasetUtils.dividir_dataset_yolo(DOWNLOAD_YOLO_DATASET_FOLDER, (0.8, 0.2, 0.0))

Moviendo archivos: 100%|██████████| 84/84 [00:00<00:00, 488.71archivo/s]
2025-05-19 16:45:31,119 - root - INFO - dividir_dataset_yolo - División del dataset YOLO completada (archivos movidos y carpetas 'full' eliminadas).


## Entrenamiento del modelo

Creamos el modelo de detección de objetos YOLOv11:

<small><em>https://docs.ultralytics.com/modes/train/#train-settings</em></small>

In [35]:
model_name = "yolo11n"
model_path = model_name + ".pt"
project_name = f"deteccion_palmeras_{model_name}_{str(uuid.uuid4()).split('-')[-1]}"

model = YOLO(model_path)
LOGGER.info(f"Ejecutando el entrenamiento del modelo {model_name} en el proyecto {project_name}")

2025-05-19 17:05:03,524 - root - INFO - <module> - Ejecutando el entrenamiento del modelo yolo11n en el proyecto deteccion_palmeras_yolo11n_0504ded7023e


In [36]:
results = model.train(
    data=str(DOWNLOAD_YOLO_DATASET_FOLDER / "dataset.yaml"),
    epochs=1,
    batch=2,
    device=0,
    imgsz=4096,
    project=project_name,
    seed=CONFIG["seed"],
    exist_ok=True,
    save_period=1,
    patience=10,
    # cache=True,
    # plots=True,
)

Ultralytics 8.3.140  Python-3.13.3 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4080 SUPER, 16376MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=2, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=downloads\yolo_dataset\dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=4096, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspective=0.0, plots=True, p

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x000001C19714EA20>
Traceback (most recent call last):
  File "e:\Documentos\Git Repositories\uba-ceia-proy-final\.venv\Lib\site-packages\torch\utils\data\dataloader.py", line 1663, in __del__
    self._shutdown_workers()
  File "e:\Documentos\Git Repositories\uba-ceia-proy-final\.venv\Lib\site-packages\torch\utils\data\dataloader.py", line 1621, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'


YOLO11n summary: 181 layers, 2,590,035 parameters, 2,590,019 gradients, 6.4 GFLOPs

Transferred 448/499 items from pretrained weights
Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed 
train: Fast image access  (ping: 0.00.0 ms, read: 27.17.6 MB/s, size: 5008.3 KB)


train: Scanning E:\Documentos\Git Repositories\uba-ceia-proy-final\ceia-proyecto-final\modulo-IA\deteccion_palmeras\downloads\yolo_dataset\labels\train.cache... 67 images, 0 backgrounds, 0 corrupt: 100%|██████████| 67/67 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.00.0 ms, read: 32.214.1 MB/s, size: 5765.5 KB)


...

Plotting labels to deteccion_palmeras_yolo11n_0504ded7023e\train\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 4096 train, 4096 val
Using 8 dataloader workers
Logging results to deteccion_palmeras_yolo11n_0504ded7023e\train
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1      40.2G      2.047      43.19      1.972          1       4096: 100%|██████████| 34/34 [37:53<00:00, 66.86s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [01:10<00:00, 14.04s/it]


                   all         17         77    0.00157      0.104    0.00205    0.00113

1 epochs completed in 0.652 hours.
Optimizer stripped from deteccion_palmeras_yolo11n_0504ded7023e\train\weights\last.pt, 7.5MB
Optimizer stripped from deteccion_palmeras_yolo11n_0504ded7023e\train\weights\best.pt, 7.5MB

Validating deteccion_palmeras_yolo11n_0504ded7023e\train\weights\best.pt...
Ultralytics 8.3.140  Python-3.13.3 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4080 SUPER, 16376MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:19<00:00,  3.90s/it]


                   all         17         77    0.00157      0.104    0.00198    0.00106
Speed: 3.9ms preprocess, 1115.2ms inference, 0.0ms loss, 24.6ms postprocess per image
Results saved to deteccion_palmeras_yolo11n_0504ded7023e\train


## Validación del modelo

Cargamos el mejor modelo:

In [5]:
project_name = "deteccion_palmeras_yolo11n_0504ded7023e"
print(f"Nombre del proyecto: {project_name}")
best_model_path = Path(project_name) / "train" / "weights" / "best.pt"

if not best_model_path.exists():
    LOGGER.error(f"El modelo {best_model_path} no existe")
    sys.exit(1)
best_model = YOLO(best_model_path)

Nombre del proyecto: deteccion_palmeras_yolo11n_0504ded7023e


Limpiamos la GPU:

In [10]:
GPUUtils.clean_gpu_usage()

Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  3% | 52% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 | 14% | 32% |


Validamos el modelo:

In [12]:
metrics = best_model.val(
    project=project_name,
    data=str(DOWNLOAD_YOLO_DATASET_FOLDER / "dataset.yaml"),
    batch=2,
    imgsz=4096,
    device=0,
)

Ultralytics 8.3.140  Python-3.13.3 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4080 SUPER, 16376MiB)
WARNING val: Slow image access detected (ping: 15.79.2 ms, read: 70.17.2 MB/s, size: 5359.4 KB). Use local storage instead of remote/mounted storage for better performance. See https://docs.ultralytics.com/guides/model-training-tips/


val: Scanning E:\Documentos\Git Repositories\uba-ceia-proy-final\ceia-proyecto-final\modulo-IA\deteccion_palmeras\downloads\yolo_dataset\labels\val.cache... 17 images, 0 backgrounds, 0 corrupt: 100%|██████████| 17/17 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:32<00:00,  3.60s/it]


                   all         17         77    0.00157      0.104    0.00252    0.00108
Speed: 36.7ms preprocess, 1317.7ms inference, 0.0ms loss, 24.8ms postprocess per image
Results saved to deteccion_palmeras_yolo11n_0504ded7023e\val


Imprimimos el reporte de validación:

In [13]:
metrics.summary()

[{'class_name': 'palmera',
  'box-p': np.float64(0.001568627450980392),
  'box-r': np.float64(0.1038961038961039),
  'box-f1': np.float64(0.0030905930075333177),
  'box-map': np.float64(0.0010810674767410837),
  'box-map50': np.float64(0.0025192620806218343),
  'box-map75': np.float64(0.0005090611054923546)}]

## Probamos la predicción

Cargamos el modelo:

In [14]:
print(f"Nombre del proyecto: {project_name}")
best_model_path = Path(project_name) / "train" / "weights" / "best.pt"

if not best_model_path.exists():
    LOGGER.error(f"El modelo {best_model_path} no existe")
    sys.exit(1)
best_model = YOLO(best_model_path)

Nombre del proyecto: deteccion_palmeras_yolo11n_0504ded7023e


### Parche

Descargamos el parche:

In [15]:
patch_name = "8deOctubreyCentenario-EspLibreLarranaga_20190828_dji_pc_5cm_patch_0"
Path(DOWNLOAD_PREDICTION_FOLDER).mkdir(parents=True, exist_ok=True)
patch_file_path = DOWNLOAD_PREDICTION_FOLDER / f"{patch_name}.jpg"
ProcesadorS3 = ProcesadorS3()
ProcesadorS3.download_patch_from_minio(patch_name, patch_file_path)
LOGGER.debug(f"Se descargó el parche {patch_file_path} desde MinIO")

Realizamos la predicción:

In [17]:
results = best_model(patch_file_path)


image 1/1 e:\Documentos\Git Repositories\uba-ceia-proy-final\ceia-proyecto-final\modulo-IA\deteccion_palmeras\downloads\predictions\8deOctubreyCentenario-EspLibreLarranaga_20190828_dji_pc_5cm_patch_0.jpg: 4096x4096 (no detections), 53.7ms
Speed: 46.5ms preprocess, 53.7ms inference, 1.2ms postprocess per image at shape (1, 3, 4096, 4096)


Obtenemos los resultados:

In [19]:
for result in results:
    xywh = result.boxes.xywh  # center-x, center-y, width, height
    xywhn = result.boxes.xywhn  # normalized
    xyxy = result.boxes.xyxy  # top-left-x, top-left-y, bottom-right-x, bottom-right-y
    xyxyn = result.boxes.xyxyn  # normalized
    names = [result.names[cls.item()] for cls in result.boxes.cls.int()]  # class name of each box
    confs = result.boxes.conf  # confidence score of each box

    print(f"Resultados para el parche: {patch_name}")
    print(f"Cantidad de detecciones: {len(names)}")
    if len(names) == 0:
        print("No se detectaron objetos.")
    else:
        for i, (name, conf, box) in enumerate(zip(names, confs, xyxy)):
            print(f"Detección {i+1}: Clase={name}, Confianza={conf.item():.2f}, Caja={box.tolist()}")

Resultados para el parche: 8deOctubreyCentenario-EspLibreLarranaga_20190828_dji_pc_5cm_patch_0
Cantidad de detecciones: 0
No se detectaron objetos.


### Imagen

In [ ]:
image_name = "8deOctubreyCentenario-EspLibreLarranaga_20190828_dji_pc_5cm_patch_0"
Path(DOWNLOAD_PREDICTION_FOLDER).mkdir(parents=True, exist_ok=True)
image_name_path = DOWNLOAD_PREDICTION_FOLDER / f"{image_name}.jpg"
ProcesadorS3 = ProcesadorS3()
ProcesadorS3.download_patch_from_minio(image_name, image_name_path)
LOGGER.debug(f"Se descargó el parche {image_name_path} desde MinIO")